In [1]:
import ee 
import geemap
geemap.ee_initialize()

# Exporting images

In [2]:
Map = geemap.Map()

image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318').select(
    ['B5', 'B4', 'B3']
)

vis_params = {'min': 0, 'max': 0.5, 'gamma': [0.95, 1.1, 1]}

Map.centerObject(image, 8)
Map.addLayer(image, vis_params, 'Landsat')
Map

Map(center=[37.47164678275328, -122.14450014746849], controls=(WidgetControl(options=['position', 'transparent…